In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

In [2]:
def get_text_or_unknown(soup, tag, attrs):
    try:
        return soup.find(tag, attrs).get_text()
    except:
        return "unknown"

def get_news(query, n_page):
    news_df = pd.DataFrame(columns=["Title", "Press", "Writer", "Article", "datetime"])
    url_query = quote(query)
    idx = 0
    for i in range(n_page):
        url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={url_query}&sort=0&photo=0&field=0&pd=0&ds=&de=&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:all,a:all&start={i * 10 + 1}"
        page = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(page, "html.parser")
        links = soup.find_all("div", {"class": "info_group"})
        
        for link in links:
            container = link.find_all("a", {"class": "info"})
            try:
                news_url = container[1]
            except IndexError:
                continue
            else:
                news_soup = BeautifulSoup(urllib.request.urlopen(news_url.get("href")).read(), "html.parser")
                title = get_text_or_unknown(news_soup, "h2", {"id": "title_area"})
                writer = get_text_or_unknown(news_soup, "em", {"class": "media_end_head_journalist_name"})
                article = get_text_or_unknown(news_soup, "article", {"id": "dic_area"})
                datetime = get_text_or_unknown(news_soup, "span", {"class": "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"})
            finally:
                press = container[0].get_text()
            news_df.loc[idx] = [title, press, writer, article, datetime]
            idx += 1

    return news_df

In [3]:
query = input("어떤 질문: ")
get_news("인공지능", 50)

,Title,Press,Writer,Article,datetime
0,"인공지능 로봇, CEO 됐다",한국경제TV언론사 선정,임원식 기자,"\n\n\n\n\n폴란드 주류업체 '딕타도르'가 최초로 인공지능, AI가 탑재된 휴...",2023.11.06. 오전 9:37
1,"SK텔레콤, 국내 인공지능 스타트업 두 곳에 94억원 투자",헤럴드경제,김현일 기자,"\nLLM 솔루션사 ‘올거나이즈’, 생명공학 기업 ‘임프리메드’SKT 주도 ‘K-A...",2023.11.06. 오전 9:02
2,"한기대, 인공지능 서비스개발 경진대회서 최우수상",더팩트,김경동 기자,\n산업경영학부 'AI 기반 군중 밀집 예방 및 관리시스템' 제안\n\n\n\n한국...,2023.11.06. 오후 2:38
3,"“안전하고 투명한 인공지능 모델, AI로 평가한다”",경향신문,주영재 기자,\n[주간경향] 인공지능(AI) 기술이 진짜와 가짜를 구분하기 어려울 정도로 정교하...,2023.11.06. 오전 8:31
4,동의대 인공지능그랜드ICT연구센터 기업가정신 포럼 특강,부산일보,unknown,\n\n\n\n\n동의대 인공지능그랜드ICT연구센터(센터장 정석찬)는 지난 10월 ...,2023.11.06. 오후 3:01
...,...,...,...,...,...
223,"첨예한 생성형 인공지능 이해관계 대립, 타협점은…",아시아경제,이종길 기자,\n문체부 26일 서울 저작권 포럼 개최전문가 대거 초청해 규범 정리 해법 모색문화...,2023.10.26. 오전 9:28
224,"스텔라앤, 인공지능 도입으로 웹툰 번역 속도 200% 성장",아시아경제,unknown,\n\n\n\n\n웹툰 번역 전문기업 스텔라앤에서 인공지능 솔루션을 도입해 2023...,2023.10.31. 오전 9:00
225,"[특별기고] 인공지능, 그 기회와 도전엔 국경이 없다",중앙일보,unknown,\n \n\n\n\n리시 수낵 영국 총리 가까운 미래에 인공지능...,2023.10.31. 오전 12:38
226,생물 종 판별에 인공지능 활용... 전문인력 양성 연찬회,오마이뉴스,unknown,"\n국립생물자원관, 25일 경북대학교에서 개최... 12개 대학원생과 4개부처 연구...",2023.10.24. 오후 3:08
